In [1]:
# Script: Optimal_POV.ipynb

In [2]:
# Step 1. Import Python Packages
import pandas as pd
import numpy as np
#import statistics as st
import scipy
#from scipy import stats
import scipy.optimize as spo
from scipy.optimize import minimize
from scipy.optimize import fsolve

In [3]:
# Step 2. Define User Input
import os
current_working_directory = os.getcwd()

InputPath = current_working_directory + "\\"
OutputPath = current_working_directory + "\\"
print("InputPath: {0}, OutputPath: {1}".format(InputPath, OutputPath))

# Output File
OutputOptPovFile="Optimal_POV.csv"

# Stock
Stock='RLK'

# Risk Aversion
Lambda = 0.50

# Order Characteristics
Size = 0.10
Volatility = 0.35

# MI Parameters
a1 = 882.900282
a2 = 0.353976
a3 = 0.755553
a4 = 0.824951
b1 = 0.963963


InputPath: C:\Users\nburg\Desktop\MMT04\, OutputPath: C:\Users\nburg\Desktop\MMT04\


In [4]:
# Step 3. Trade Cost Functions

# Market Impact
def MI_calc(a1, a2, a3, a4, b1, Size, Volatility, POV):
    mi = (a1*Size**a2*Volatility**a3) * (b1*POV**a4 + (1-b1))
    return mi

# Timing Risk
def TR_calc(Size, Volatility, POV):
    tr = Volatility * ((1/3*1/250*Size*(1-POV)/POV )**0.5) * 10**4
    return tr


# Price Appreciation
def PA_calc(Side, AlphaBp, Size, POV):
    pa = Side * (1/2) * AlphaBp * Size * ((1 - POV)/POV)
    return pa

# Trade Time
def TradeTime_calc(Size, POV):
    tt = Size * ((1-POV)/POV)
    return tt


In [5]:
# Step 4. POV Optimization Function

def Opt_POV_calc(POV, Size, Volatility, Lambda, a1, a2, a3, a4, b1):
    MI = (a1*Size**a2*Volatility**a3) * (b1*POV**a4 + (1-b1))
    TR = Volatility * ((1/3*1/250*Size*(1-POV)/POV )**0.5) * 10**4
    f = MI + (Lambda * TR)
    return f
    

In [6]:
# Step 5. POV Optimization

bound1 = [(0.00001,1)]
bounds = (bound1)

# Run POV Optimization
POV = minimize(Opt_POV_calc, 0.5, bounds = bounds, args = (Size, Volatility, Lambda, a1, a2, a3, a4, b1))

# Get POV value
#POV = pd.Series(POV.x.tolist()[0])
#POV=POV.x
#POV

#POV.x.tolist()[0]
POV = pd.Series(POV.x.tolist()[0])
#POV

In [7]:
# Step 6. Get Trade Time

TradeTime = TradeTime_calc(Size, POV)
print(POV[0], TradeTime[0])


0.14545770930767105 0.587485046175729


In [8]:
# Step 7. Save Results to Output csv file

#POV = pd.Series(POV.x.tolist()[0])
pov = POV[0]
tradetime = TradeTime[0]
Lambda = pd.Series(Lambda)
Size = pd.Series(Size)
Volatility = pd.Series(Volatility)
a1 = pd.Series(a1)
a2 = pd.Series(a2)
a3 = pd.Series(a3)
a4 = pd.Series(a4)
b1 = pd.Series(b1)

print(pov, tradetime)

# Output File Data
POV_df = pd.concat([POV, TradeTime, Lambda, Size, Volatility, a1, a2, a3, a4, b1], axis =1)
POV_df.columns = [['POV', 'TradeTime', 'Lambda', 'Size', 'Volatility', 'a1', 'a2', 'a3', 'a4', 'b1']]
POV_df = POV_df.transpose()
POV_df.columns = ['Value']
POV_df.index.rename('Parameter', inplace=True)

# Save File
POV_df.to_csv(OutputPath+OutputOptPovFile, index = True, header = True)

0.14545770930767105 0.587485046175729


In [9]:
# To write output file
POV_df = pd.concat([POV, TradeTime, Lambda, Size, Volatility, a1, a2, a3, a4, b1], axis =1)
POV_df.columns = [['POV', 'TradeTime', 'Lambda', 'Size', 'Volatility', 'a1', 'a2', 'a3', 'a4', 'b1']]
POV_df = POV_df.transpose()
POV_df.columns = ['Value']
POV_df.index.rename('Parameter', inplace=True)



In [10]:
#POV_df.to_csv(OutputPath+'Algo-SS-Optimal-POV.csv', index = True, header = True)
POV_df.to_csv(OutputPath+OutputOptPovFile, index = True, header = True)
